# Лабораторная работа №1

### Выполнена студентом группы БИ-17-2 Якубовым Артёмом

#### Ссылка на гитхаб: https://github.com/Aranteus/Python/tree/master/Machine%20Learning%20Jupyter%20Notebook

## Задание №1

##### Считать файлы из формата CSV. Необходимо правильно настроить разделитель между ячейками и десятичный разделитель.

Сначала необходимо импортировать библиотеки, которые помогут при выполнении поставленной задачи.

In [853]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import math

Загружаем 1-ый файл и просматриваем его.

In [854]:
data_person = pd.read_csv("CASC_Constant.csv")
print(data_person.head())

   CustomerId ActivationDate   Age     Sex  SubscribedEmail  SubscribedPush
0     2728183     2015-01-01  21.0  Female            False            True
1     2728198     2015-01-01  21.0  Female             True            True
2     2728306     2015-01-01  21.0  Female             True            True
3     2728178     2015-01-01  22.0    Male             True            True
4     2728322     2015-01-01  22.0    Male             True            True


In [855]:
data_transact = pd.read_csv("casc-resto.csv", sep = ';', decimal = ',')
print(data_transact.head())

   CustomerID  Restaurant      RKDate                    RegionName  \
0     2898197         391  2017-07-16  Москва и Московская область    
1     2903215          43  2015-04-07  Москва и Московская область    
2     2748887          43  2015-05-22  Москва и Московская область    
3     2862077          46  2015-03-05  Москва и Московская область    
4     2862077          46  2015-03-11  Москва и Московская область    

   BrandsNames DishCategoryName  Quantity  SummBasic  SummAfterPointsUsage  
0  TGI FRIDAYS      NON ALCOHOL         2        2.0                   2.0  
1     IL Патио      NON ALCOHOL         1        5.0                   5.0  
2     IL Патио      NON ALCOHOL         1        5.0                   5.0  
3     IL Патио      NON ALCOHOL         1        5.0                   5.0  
4     IL Патио      NON ALCOHOL         1        5.0                   5.0  


Загружаем 2-ой файл и просматриваем его.

In [856]:
#data_transact = pd.read_csv("casc-resto-small.csv")
#print(data_transact.head())

Используем описательную статистику для первичного анализа данных.

In [857]:
print(data_transact.describe())
print(data_transact.info())

         CustomerID     Restaurant       Quantity      SummBasic  \
count  8.822220e+05  882222.000000  882222.000000  882222.000000   
mean   2.809199e+06     468.268804       1.154900     287.725795   
std    6.178420e+04     279.170967       0.650287     264.555291   
min    2.728046e+06      40.000000       0.000000       0.500000   
25%    2.754886e+06     333.000000       1.000000     120.000000   
50%    2.785104e+06     434.000000       1.000000     225.000000   
75%    2.878900e+06     712.000000       1.000000     379.000000   
max    2.913132e+06     980.000000     107.000000   29450.000000   

       SummAfterPointsUsage  
count         882222.000000  
mean             260.255589  
std              251.289517  
min            -2593.000000  
25%              110.000000  
50%              199.000000  
75%              348.940000  
max            29450.000000  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 882222 entries, 0 to 882221
Data columns (total 9 columns):
Custome

## Задание №2

#### Для каждого клиента по транзакционным данным необходимо:

1. Зафиксировать дату 2017-07-01 как дату, на которую считаются характеристики клиентов.

2. Для каждого клиента определить, совершал ли он визит в период между 2017-07-01 и 2017-12-31 включительно. Полученный флаг является зависимой переменной Y, которую требуется предсказывать

3. Для каждого клиента расчитать Recency (количество дней до последнего визита клиента перед зафиксированной датой 2017-07-01), Frequency (средняя частота походов клиентов на дату 2017-07-01, измеряется как среднее количество визитов в месяц [месяц полагать равным 30 дням] за время между 2017-07-01 и первым визитом клиента) и Monetary Value (средний чек клиента по всем покупкам до 2017-07-01). Необходимо помнить, что данные являются реальными данными по москвоским ресторанам. Необходимо проверить на адекватность получаемые значения. Например, не может быть средний чек большого количества клиентов быть меньше 200 рублей, либо частота визитов типичного клиента равно 20 посещениям в месяц.

4. Для каждого клиента студенту необходимо самостоятельно придумать и расчитать характеристику клиента по транзакционным данным, которая могла бы помочь предсказывать зависимую переменную Y. Необходимо пояснить, почему выбрана именно эта характеристика.

Все задания этого пункта выполняются через «pandas.DataFrame.groupby», т.е. циклы, лямбды и прочие цикло-образные конструкции не должны быть основным способом получения результата по каждому из пунктов (или подпунктов).

Пытаюсь понять, чем отличается поле SummAfterPointsUsage от SummBasic, предполагая, что это как-то связано с количеством продуктов.
Пришёл к выводу, что общая цена за транзакцию оценивается как произведение количества еды на сумму со скидкой (последний столбец).

In [858]:
data_transact[(data_transact.SummAfterPointsUsage != data_transact.SummBasic) & (data_transact.Quantity != 1)].head()

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage
212,2758212,721,2015-06-17,Москва и Московская область,"IL Патио,Планета Суши",FOOD,5,9.0,5.07
222,2750233,40,2016-11-18,Москва и Московская область,Американский Бар и Гриль,FOOD,2,10.0,5.00
226,2730340,43,2015-01-03,Москва и Московская область,IL Патио,NON ALCOHOL,2,10.0,9.86
236,2888593,43,2015-05-24,Москва и Московская область,IL Патио,NON ALCOHOL,2,10.0,5.10
246,2733081,43,2015-08-16,Москва и Московская область,IL Патио,FOOD,2,10.0,6.26


Окончательно убеждаюсь в своих доводах, что в последнем столбце указана цена со скидкой.

In [859]:
data_transact[data_transact.SummAfterPointsUsage > data_transact.SummBasic]

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage
421338,2745922,752,2017-04-29,Санкт-Петербург и Ленинградская область,"IL Патио,Планета Суши",FOOD,1,359.0,359.03


Меняю тип столбца RKDate со строкового на дату, чтобы было легче с ней работать, и проверяю, что изменения прошли успешно.

In [860]:
data_transact.RKDate = pd.to_datetime(data_transact.RKDate)
print(data_transact.head())
type(data_transact.RKDate[0])

   CustomerID  Restaurant     RKDate                    RegionName  \
0     2898197         391 2017-07-16  Москва и Московская область    
1     2903215          43 2015-04-07  Москва и Московская область    
2     2748887          43 2015-05-22  Москва и Московская область    
3     2862077          46 2015-03-05  Москва и Московская область    
4     2862077          46 2015-03-11  Москва и Московская область    

   BrandsNames DishCategoryName  Quantity  SummBasic  SummAfterPointsUsage  
0  TGI FRIDAYS      NON ALCOHOL         2        2.0                   2.0  
1     IL Патио      NON ALCOHOL         1        5.0                   5.0  
2     IL Патио      NON ALCOHOL         1        5.0                   5.0  
3     IL Патио      NON ALCOHOL         1        5.0                   5.0  
4     IL Патио      NON ALCOHOL         1        5.0                   5.0  


pandas._libs.tslibs.timestamps.Timestamp

Сортирую данные по дате и узнаю последнюю дату.

In [861]:
data_transact = data_transact.sort_values("RKDate")
data_transact.tail()

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage
684746,2898779,354,2017-12-31,Москва и Московская область,Costa Coffee,КОМИССИЯ МОРОЖ,1,99.0,99.00
684747,2898779,354,2017-12-31,Москва и Московская область,Costa Coffee,КОМИССИЯ МОРОЖ,1,99.0,99.00
833588,2728838,440,2017-12-31,Москва и Московская область,TGI FRIDAYS,FOOD,1,430.0,430.00
667073,2821630,463,2017-12-31,Москва и Московская область,TGI FRIDAYS,FOOD,1,599.0,422.36
580103,2907562,553,2017-12-31,Казань,IL Патио,FOOD,1,595.0,595.00


Узнаю, есть ли данные на фиксированной дате.

In [862]:
fix_date = pd.to_datetime('2017-07-01')
data_transact[data_transact.RKDate == fix_date].head()

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage
269087,2831289,93,2017-07-01,Москва и Московская область,IL Патио,FOOD,1,65.0,48.73
253001,2887910,925,2017-07-01,Москва и Московская область,"IL Патио,Планета Суши",FOOD,1,435.0,326.18
168009,2811579,727,2017-07-01,Москва и Московская область,IL Патио,FOOD,1,385.0,288.77
382746,2887910,925,2017-07-01,Москва и Московская область,"IL Патио,Планета Суши",NON ALCOHOL,1,219.0,164.21
364392,2905704,295,2017-07-01,Москва и Московская область,Шикари,FOOD,1,155.0,116.19


Узнаю, есть ли данные после фиксированной даты.

In [863]:
data_transact[data_transact.RKDate >= fix_date].head()

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage
269087,2831289,93,2017-07-01,Москва и Московская область,IL Патио,FOOD,1,65.0,48.73
253001,2887910,925,2017-07-01,Москва и Московская область,"IL Патио,Планета Суши",FOOD,1,435.0,326.18
168009,2811579,727,2017-07-01,Москва и Московская область,IL Патио,FOOD,1,385.0,288.77
382746,2887910,925,2017-07-01,Москва и Московская область,"IL Патио,Планета Суши",NON ALCOHOL,1,219.0,164.21
364392,2905704,295,2017-07-01,Москва и Московская область,Шикари,FOOD,1,155.0,116.19


Создаю новую переменную для содержания в ней данных о транзакциях между фиксированной датой и концом года для того, чтобы узнать, совершались ли в этот период транзакции для каждого клиента.

In [864]:
rkdate = data_transact.RKDate[(data_transact.RKDate >= fix_date) & (data_transact.RKDate <= pd.to_datetime('2017-12-31'))]

Создаём группу для того, чтобы узнать какие клиенты и сколько раз посещали ресторан в последние полгода.

In [865]:
gb = data_transact.groupby(["CustomerID", rkdate]).count()['Restaurant'].to_frame().reset_index()
gb

,CustomerID,RKDate,Restaurant
0,2728089,2017-07-02,6
1,2728107,2017-11-05,7
2,2728135,2017-08-09,3
3,2728135,2017-10-21,11
4,2728135,2017-12-28,11
...,...,...,...
18250,2913123,2017-12-12,5
18251,2913123,2017-12-26,4
18252,2913123,2017-12-27,4
18253,2913132,2017-08-03,7


Достаём уникальные ID из датафрейма и отсортируем их.

In [866]:
data = pd.DataFrame()
data["Customer"] = data_transact.CustomerID.unique()
data = data.sort_values("Customer")

Достаём ID клиентов, посещавших рестораны в последние полгода из группы, и присваиваем им 1 в датасете со всеми клиентами. Проверяем, что все клиенты отметились.

In [867]:
c_values = gb.CustomerID.values
data["Y"] = [1 if data["Customer"][i] in c_values else 0 for i in range(len(data))]
print(len(data[data.Y == True]))
print(len(data[data.Y == False]))
print(len(data))

4644
5356
10000


Достаём ID клиентов с последним посещением ресторана до фиксированной даты. Превращаем разницу дат в дни и записываем в датафрейм.

In [868]:
new_df = data_transact[["CustomerID", "RKDate"]]
new_df = new_df[new_df["RKDate"] < fix_date]
group = new_df.groupby("CustomerID").max().reset_index()
g_values = group.CustomerID.values
data["Recency"] = [(fix_date - group["RKDate"][i]).days if data["Customer"][i] in g_values else 0 for i in range(len(data))]
data["Recency"]

29      160
13      720
18       15
6       177
16      115
       ... 
7395      0
7441      0
7417      0
7394      0
9724      0
Name: Recency, Length: 10000, dtype: int64

Считаем количество посещений ресторана для каждого клиента в каждом месяце каждого года. Высчитываем среднее значение для каждой группы и записываем в датафрейм.

In [869]:
df_freq = data_transact[["CustomerID", "RKDate", "Restaurant"]]
df_freq = df_freq[df_freq["RKDate"] < fix_date]
group_count = df_freq.groupby(["CustomerID", df_freq["RKDate"].dt.year, df_freq["RKDate"].dt.month, "Restaurant"]).count()
group_mean = group_count.groupby(["CustomerID"])["RKDate"].mean().reset_index()
g_values = group_mean.CustomerID.values
data["Frequency"] = [group_mean["RKDate"][i] if data["Customer"][i] in g_values else 0 for i in range(len(data))]
print(data["Frequency"])

29       6.333333
13      11.333333
18       5.500000
6        8.250000
16       5.250000
          ...    
7395     0.000000
7441     0.000000
7417     0.000000
7394     0.000000
9724     0.000000
Name: Frequency, Length: 10000, dtype: float64


Считаем сумму транзакций для клиентов и записываеи в датафрейм.

In [870]:
data_monetary = data_transact[["CustomerID", "SummAfterPointsUsage", "Quantity", "RKDate", "Restaurant"]]
data_monetary = data_monetary[data_monetary.RKDate < fix_date]
data_monetary["Sum"] = data_monetary.SummAfterPointsUsage * data_monetary.Quantity
data_monetary = data_monetary[["CustomerID", "Sum", "Restaurant", "RKDate"]]
group_max = data_monetary.groupby(["CustomerID", "Restaurant", "RKDate"])["Sum"].sum().reset_index()
group_money = group_max.groupby(["CustomerID"])["Sum"].mean().reset_index()
m_values = group_money.CustomerID.values
data["Monetary Value"] = [group_money["Sum"][i] if data["Customer"][i] in m_values else 0 for i in range(len(data))]
data.head()

,Customer,Y,Recency,Frequency,Monetary Value
29,2728046,0,160,6.333333,1743.555556
13,2728088,1,720,11.333333,2377.000000
18,2728089,0,15,5.500000,3441.200000
6,2728095,0,177,8.250000,2572.910000
16,2728107,0,115,5.250000,3227.610000


Считаем максимальную сумму транзакции для каждого клиента, потому что она возможно может повлиять на посещение людьми ресторанов.

In [871]:
group_max = group_max.groupby(["CustomerID"])["Sum"].max().reset_index()
max_values = group_max.CustomerID.values
data["Max Value"] = [group_max["Sum"][i] if data["Customer"][i] in max_values else 0 for i in range(len(data))]
data.head()

,Customer,Y,Recency,Frequency,Monetary Value,Max Value
29,2728046,0,160,6.333333,1743.555556,4230.0
13,2728088,1,720,11.333333,2377.000000,4310.0
18,2728089,0,15,5.500000,3441.200000,8658.0
6,2728095,0,177,8.250000,2572.910000,5530.0
16,2728107,0,115,5.250000,3227.610000,4790.0


## Задание №3

#### Для каждого клиента к уже рассчитанным характеристикам необходимо из второй таблицы притянуть данные по полу и возрасту клиента.

In [872]:
data.rename(columns={'Customer':'CustomerId'}, inplace=True)
print(data.head())
full_data = pd.merge(data, data_person, how="inner", on="CustomerId")

    CustomerId  Y  Recency  Frequency  Monetary Value  Max Value
29     2728046  0      160   6.333333     1743.555556     4230.0
13     2728088  1      720  11.333333     2377.000000     4310.0
18     2728089  0       15   5.500000     3441.200000     8658.0
6      2728095  0      177   8.250000     2572.910000     5530.0
16     2728107  0      115   5.250000     3227.610000     4790.0


Удаляем строки с пропущенными данными и возвращаем удалённые индексы.

In [873]:
full_data = full_data.dropna()
full_data = full_data.reset_index(drop=True)

Заменяем строковые значения пола на булевы. 

In [874]:
full_data = full_data.replace({'Sex': {'Male': 1,'Female': 0}})
full_data.describe()

,CustomerId,Y,Recency,Frequency,Monetary Value,Max Value,Age,Sex
count,8.856000e+03,8856.000000,8856.000000,8856.000000,8856.000000,8.856000e+03,8856.000000,8856.000000
mean,2.812411e+06,0.462737,257.350384,6.936896,2236.377655,7.247464e+03,32.814476,0.420845
std,6.226241e+04,0.498638,254.369466,4.604054,2924.324556,4.473165e+04,11.360298,0.493723
min,2.728046e+06,0.000000,0.000000,0.000000,0.000000,0.000000e+00,16.000000,0.000000
25%,2.756260e+06,0.000000,41.000000,4.625000,1188.386494,2.360750e+03,26.000000,0.000000
50%,2.795320e+06,0.000000,155.000000,6.000000,1730.011000,3.718065e+03,31.000000,0.000000
75%,2.880346e+06,1.000000,436.000000,7.833333,2521.225000,6.428033e+03,37.000000,1.000000
max,2.913132e+06,1.000000,912.000000,78.034483,117100.291818,3.342239e+06,218.000000,1.000000


Удаляем все нереальные данные.

In [875]:
full_data = full_data[full_data.Frequency < 30]
full_data = full_data[full_data["Monetary Value"] < 10000]
full_data = full_data[full_data["Max Value"] < 100000]
full_data = full_data[full_data.Recency > 0]
full_data = full_data[full_data.Frequency > 0]
full_data = full_data[full_data["Monetary Value"] > 0]
full_data = full_data[full_data["Max Value"] > 0]
full_data = full_data[full_data.Age < 100]
full_data.describe()

,CustomerId,Y,Recency,Frequency,Monetary Value,Max Value,Age,Sex
count,8.579000e+03,8579.000000,8579.000000,8579.000000,8579.000000,8579.000000,8579.000000,8579.000000
mean,2.811865e+06,0.458911,258.236275,6.721479,2061.362930,5758.252644,32.093834,0.421028
std,6.199364e+04,0.498338,253.611872,3.471082,1368.145688,6990.942145,8.025021,0.493753
min,2.728046e+06,0.000000,1.000000,1.000000,174.200000,215.000000,16.000000,0.000000
25%,2.756032e+06,0.000000,42.000000,4.636364,1198.583333,2368.000000,26.000000,0.000000
50%,2.793992e+06,0.000000,157.000000,5.965517,1728.500000,3709.000000,31.000000,0.000000
75%,2.879766e+06,1.000000,436.000000,7.750000,2496.429167,6343.255000,37.000000,1.000000
max,2.912502e+06,1.000000,912.000000,29.357143,9909.860000,88999.000000,83.000000,1.000000


## Задание № 4

#### Разбить клиентов случайно на две группы в соотношении 80:20. Большая группа — это обучающая выборка, а маленькая группа это тестовая выборка.

In [876]:
X = full_data[["Recency", "Frequency", "Monetary Value", "Max Value", "Age", "Sex"]]
y = full_data[["Y"]].squeeze()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Задание № 5-7

#### Обучить модель логистической регрессии на обучающей выборке.

#### Предсказать вероятность прихода на тестовой выборке.

#### Посчитать метрики Precision и Recall на основании предсказаний на тестовой выборке и истинных значений зависимой переменной Y.

In [877]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
pred = logreg.predict(X_test)
print(classification_report(y_test, pred))
print('Model score:', logreg.score(X_test, y_test))

              precision    recall  f1-score   support

           0       0.54      1.00      0.70       926
           1       1.00      0.00      0.01       790

    accuracy                           0.54      1716
   macro avg       0.77      0.50      0.35      1716
weighted avg       0.75      0.54      0.38      1716

Model score: 0.5407925407925408


C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
